In [1]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import math
import pylab
import scipy.stats as stats
%matplotlib inline

In [2]:
cols = [
    'id',
    'host_id',
    'zipcode',
    'property_type',
    'room_type',
    'accommodates',
    'bedrooms',
    'beds',
    'bed_type',
    'price',
    'number_of_reviews',
    'review_scores_rating',
    'host_listings_count',
    'availability_30',
    'minimum_nights',
    'bathrooms'
]

data = pd.read_csv('../data/listings.csv', usecols=cols)

In [3]:
data.head()

,id,host_id,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,minimum_nights,availability_30,number_of_reviews,review_scores_rating
0,109,521,1.0,90230,Condominium,Entire home/apt,6,2.0,2.0,3.0,Real Bed,$122.00,30,0,2,80.0
1,344,767,1.0,91505,House,Entire home/apt,6,1.0,3.0,3.0,Real Bed,$168.00,2,0,6,93.0
2,2708,3008,2.0,90046,Apartment,Private room,1,1.5,1.0,1.0,Real Bed,$79.00,28,0,21,98.0
3,2732,3041,2.0,90405,Apartment,Private room,1,1.0,1.0,1.0,Pull-out Sofa,$140.00,1,11,19,96.0
4,2864,3207,1.0,90706,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,$80.00,2,0,0,NaN


In [4]:
# Identify the number of missing entries in each column
# for col in data.columns:
#    print(col + ', Number of Missing Values:', len(data[col][data[col].isnull()]))

# Remove NaN values from dataframe
original = len(data)
data = data.dropna(how='any', subset=['zipcode', 'property_type', 'bedrooms', 'beds', 'bathrooms'])

# Drop any inconsistent values
data = data[data['accommodates'] != 0]
data = data[data['bedrooms'] != 0]
data = data[data['beds'] != 0]
data = data[data['price'] != 0.00]


In [5]:
# Clean price data

# Convert formatting for price
data['price'] = (data['price'].str.replace(r'[^-+\d.]', '').astype(float))

In [6]:
# Clean zipcode data

# Convert zipcode
data['zipcode'] = data['zipcode'].str.replace(r'-\d+', '')

# Ensure all zipcodes are of length 5
data = data[data['zipcode'].map(len) == 5]

In [7]:
# Clean the reviews data

# Turn NaN scores with 0 reviews into 'No Reviews'
idx_vals = data['review_scores_rating'][data['number_of_reviews'] == 0].index.values.tolist()
data.loc[idx_vals, ('review_scores_rating')] = data['review_scores_rating'][data['number_of_reviews'] == 0].replace('NaN', 'No Reviews')

# Remove NaN values
data = data[~data['review_scores_rating'].isnull()]

# Convert review_scores_rating into buckets
def convert_scores_buckets(val):
    if val == 'No Reviews':
        return 'No Reviews'
    elif val >= 95.0:
        return '95-100'
    elif val >= 90.0 and val < 95.0:
        return '90-94'
    elif val >= 85.0 and val < 90.0:
        return '85-89'
    elif val >= 80.0 and val < 85.0:
        return '80-84'
    elif val >= 70.0 and val < 80.0:
        return '70-79'
    elif val >= 60.0 and val < 70.0:
        return '60-69'
    elif val >= 50.0 and val < 60.0:
        return '50-59'
    elif val >= 40.0 and val < 50.0:
        return '40-49'
    elif val >= 30.0 and val < 40.0:
        return '30-39'
    elif val >= 20.0 and val < 30.0:
        return '20-29'
    elif val >= 10.0 and val < 20.0:
        return '10-19'
    elif val < 10.0:
        return '0-9'
    
data['review_scores_rating'] = data['review_scores_rating'].apply(convert_scores_buckets)
print('Unique Values in the Column:', np.unique(data['review_scores_rating']))

Unique Values in the Column: ['20-29' '30-39' '40-49' '50-59' '60-69' '70-79' '80-84' '85-89' '90-94'
 '95-100']


In [8]:
# Encode categorical variables

zipcode_dummies = pd.get_dummies(data['zipcode'])
property_dummies = pd.get_dummies(data['property_type'])
room_dummies = pd.get_dummies(data['room_type'])
bed_dummies = pd.get_dummies(data['bed_type'])
ratings_scores_dummies = pd.get_dummies(data['review_scores_rating'])

# Replace the old columns with new one-hot encoded columns
df = pd.concat((data.drop(['zipcode', \
    'property_type', 'room_type', 'bed_type', 'review_scores_rating'], axis=1), \
    zipcode_dummies.astype(int), property_dummies.astype(int), \
    room_dummies.astype(int), bed_dummies.astype(int), ratings_scores_dummies.astype(int)), \
    axis=1)

# Move 'price' to the end of the dataframe
cols = list(df.columns.values)
idx = cols.index('price')
rearrange_cols = cols[:idx] + cols[idx+1:] + [cols[idx]]
df = df[rearrange_cols]

In [9]:
# Convert non-categorical variables to floats and standardize
def standardize_col(col):
    mean = np.mean(col)
    std = np.std(col)
    return col.apply(lambda x: (x - mean) / std)

non_cat_vars = ['accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'host_listings_count', 'availability_30', 'minimum_nights', 'bathrooms']
for col in non_cat_vars:
    df[col] = df[col].astype(float)
    df[col] = standardize_col(df[col])


In [10]:
df.head()

,id,host_id,host_listings_count,accommodates,bathrooms,bedrooms,beds,minimum_nights,availability_30,number_of_reviews,...,30-39,40-49,50-59,60-69,70-79,80-84,85-89,90-94,95-100,price
0,109,521,-0.165599,0.794744,0.607989,0.430445,0.514129,1.533262,-1.114222,-0.616103,...,0,0,0,0,0,1,0,0,0,122.0
1,344,767,-0.165599,0.794744,-0.520603,1.438418,0.514129,-0.202409,-1.114222,-0.552203,...,0,0,0,0,0,0,0,1,0,168.0
2,2708,3008,-0.143871,-1.055204,0.043693,-0.577527,-0.661159,1.409286,-1.114222,-0.312576,...,0,0,0,0,0,0,0,0,1,79.0
3,2732,3041,-0.143871,-1.055204,-0.520603,-0.577527,-0.661159,-0.264397,-0.097982,-0.344527,...,0,0,0,0,0,0,0,0,1,140.0
5,5728,9171,-0.013506,-0.685215,-0.520603,-0.577527,-0.661159,-0.264397,-0.097982,3.936801,...,0,0,0,0,0,0,0,0,1,75.0


In [11]:
df.to_csv("listings_clean.csv",index=False)